# Creating the final dataset `residence_time.nc`

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import glob
from tqdm import tqdm
from datetime import datetime

In [2]:
def get_time_from_path(path):
    parts = path.split(os.sep)
    year = int(parts[-2][0:4])
    month = int(parts[-1].split("_")[-1][:2])
    return datetime(year, month, 15)

def calculate_mean_traveltime(traveltime_total: np.ndarray, particle_counter: np.ndarray):
    '''
    Takes two arrays and divides them (elementwise), avoiding division by 0
    
    Parameters:
        traveltime_total (arr): Array that contains the summed travel time for each particle reaching each gridpoint
        particle_counter (arr): Array that contains the number of particles that passed each gridpoint
    
    Returns:
        traveltime_mean (arr): Mean (traveltime_total/particle_counter)
    '''
    # creating mask to avoid division by 0
    mask = (particle_counter != 0)
    # create array with the same shape as the input arrays
    traveltime_mean = np.zeros_like(traveltime_total).astype(float)
    # calculate mean
    traveltime_mean[mask] = traveltime_total[mask] / particle_counter[mask]
    return traveltime_mean

In [9]:
output_base_dir = 'OUTPUT_BASE_PATH'  # Please set your output base path here
file_paths = sorted(glob.glob(os.path.join(output_base_dir, "[1-2][0-9][0-9][0-9]", "travel_times_*.nc")))

In [ ]:
%%time

# importing data
season_dict = {
    12: "DJF", 1: "DJF", 2: "DJF",
    3: "MAM", 4: "MAM", 5: "MAM",
    6: "JJA", 7: "JJA", 8: "JJA",
    9: "SON", 10: "SON", 11: "SON"
}

datasets = []

for file in tqdm(file_paths, desc='Processing Files', unit='file'):
    ds = xr.open_dataset(file)
    time = get_time_from_path(file)
    ds = ds.expand_dims({"time": [time]})
    datasets.append(ds)
    
# concatenate dataset
ds_combined = xr.concat(datasets, dim="time")
ds_combined = ds_combined.assign_coords(season=('time' , [season_dict[m] for m in ds_combined["time.month"].values]))

# shift coordinates (to represent grid cell centers)
ds_combined['lat'] = ds_combined.lat + 0.25
ds_combined['lon'] = ds_combined.lon + 0.25

In [11]:
# calculating mean travel time per month and adding it to the dataset

ds_combined['traveltime_mean'] = xr.full_like(ds_combined['traveltime_total'], fill_value=np.nan)

for time in tqdm(ds_combined.time, desc='Progressing through time', unit='timestep'):
    mean_traveltime_month = calculate_mean_traveltime(ds_combined.sel(time=time).traveltime_total.values,
                                                      ds_combined.sel(time=time).particle_counter.values)
    ds_combined['traveltime_mean'].loc[dict(time=time)] = mean_traveltime_month

Progressing through time: 100%|██████████| 999/999 [00:03<00:00, 321.73timestep/s]


In [12]:
ds_combined

<xarray.Dataset> Size: 6GB
Dimensions:           (time: 999, lat: 360, lon: 720)
Coordinates:
  * time              (time) datetime64[ns] 8kB 1941-01-15 ... 2024-03-15
  * lat               (lat) float64 3kB -89.75 -89.25 -88.75 ... 89.25 89.75
  * lon               (lon) float64 6kB -179.8 -179.2 -178.8 ... 179.2 179.8
    season            (time) <U3 12kB 'DJF' 'DJF' 'MAM' ... 'DJF' 'DJF' 'MAM'
Data variables:
    traveltime_total  (time, lat, lon) float64 2GB 4.629e+04 ... 2.034e+04
    particle_counter  (time, lat, lon) float64 2GB 31.0 73.0 64.0 ... 13.0 19.0
    traveltime_mean   (time, lat, lon) float64 2GB 1.493e+03 ... 1.071e+03

In [ ]:
# add attributes to the dataset
# coordinates
ds_combined['time'].attrs = {
    'standard_name': 'time',
    'long_name': 'Time',
    'description': 'Midpoint of Monthly Averages'
}
ds_combined['time'].encoding = {
    'units': 'days since 1941-01-15',
    'calendar': 'gregorian'
}
ds_combined['lat'].attrs = {
    'standard_name': 'latitude',
    'long_name': 'Latitude',
    'description': 'Grid Cell Centers',
    'units': 'degrees_north'}
ds_combined['lon'].attrs = {
    'standard_name': 'longitude',
    'long_name': 'Longitude',
    'description': 'Grid Cell Centers',
    'units': 'degrees_east'}
ds_combined['season'].attrs = {
    'long_name': 'Season',
    'description': 'Season associated with respective month'
}

# data variables
ds_combined['traveltime_total'].attrs = {
    'long_name': 'Total Traveltime',
    'description': 'Sum of Residence Time over each Grid Cell',
    'units': 'residence_time_hours'
}
ds_combined['particle_counter'].attrs = {
    'long_name': 'Particle Counter',
    'description': 'Number of eligible Particles over each Grid Cell'
}
ds_combined['traveltime_mean'].attrs = {
    'long_name': 'Mean Traveltime',
    'description': 'Average of Travel Time until particle reach grid cell',
    'units': 'residence_time_hours'
}

# general attributes
ds_combined.attrs = {
    "title": "Travel Time and Particle Data from FLEXPART",
    "summary": "This dataset contains travel time and particle count statistics aggregated on a grid.",
    "institution": "University of Vienna",
    "source": "FLEXPART v11",
    "history": "Created on 2025-03-09 using xarray and Python",
    "Conventions": "CF-1.10"
}

In [14]:
# adding time_bnds
time = pd.to_datetime(ds_combined['time'].values)

# create bounds
start_dates = pd.to_datetime(time.to_period('M').start_time)
end_dates = (start_dates + pd.offsets.MonthBegin(1))

# stack bounds
time_bounds = xr.DataArray(
    np.stack([start_dates, end_dates], axis=1),
    dims=["time", "nbounds"]
)

# Adding to dataset
ds_combined["time_bnds"] = time_bounds
ds_combined['time'].attrs["bounds"] = "time_bnds"

In [16]:
ds_combined.to_netcdf("residence_time.nc", encoding={
    "traveltime_total": {"dtype": "float64"},
    "traveltime_mean": {"dtype": "float64"},
    "particle_counter": {"dtype": "float64"},
    "time": {
        "units": "days since 1940-01-01",
        "calendar": "gregorian"
    },
    "time_bnds": {
        "units": "days since 1940-01-01",
        "calendar": "gregorian"
    }
})